In [28]:
import pandas as pd
import re

def extract_quantity(text):
    # Se o texto for nulo, retorne None
    if pd.isnull(text):
        return None
    
    # Expressão regular para encontrar "quantidade + unidade" no início do texto, incluindo formatos de soma
    quantity_unit_pattern = re.compile(r'^((?:\([\d+,\.\s]+\)\s*|[0-9,\.]+\s*)\w+)')
    quantity_unit_match = quantity_unit_pattern.match(text)
    if quantity_unit_match:
        quantity_unit = quantity_unit_match.group(1).strip()  # Pegue a primeira quantidade e unidade
    else:
        quantity_unit = None

    return quantity_unit

def main():
    df = pd.read_excel("new_gov.xlsx")
    df['Quantidade'] = df['APRESENTAÇÃO'].apply(extract_quantity)
    df_output = df[['APRESENTAÇÃO', 'Quantidade']]
    df_output.to_csv("output.csv", index=False)

if __name__ == "__main__":
    main()


In [29]:
import pandas as pd
import re

def extract_quantity_product(text):
    if pd.isnull(text):
        return None, None

    quantity_unit_pattern = re.compile(r'^((?:\([\d+,\.\s]+\)\s*|[0-9,\.]+\s*)\w+)')
    quantity_unit_match = quantity_unit_pattern.match(text)
    if quantity_unit_match:
        quantity_unit = quantity_unit_match.group(1).strip()
        remaining_text = text.replace(quantity_unit, '', 1).strip()  # Remover a quantidade e unidade
    else:
        quantity_unit = None
        remaining_text = text

    # Encontrar o tipo de produto como a primeira palavra ou frase após a quantidade e unidade
    product_pattern = re.compile(r'^(\w+(?:\s+\w+)*)')  
    product_match = product_pattern.match(remaining_text)
    if product_match:
        product = product_match.group(1).strip()
        remaining_text = remaining_text.replace(product, '', 1).strip()  # Remover o tipo de produto
    else:
        product = None

    return quantity_unit, product, remaining_text  # Detalhes da embalagem é o texto restante

def main():
    df = pd.read_excel("new_gov.xlsx")
    df[['Quantidade', 'Tipo de Produto', 'Detalhes da Embalagem']] = df['APRESENTAÇÃO'].apply(
        lambda x: pd.Series(extract_quantity_product(x))
    )
    df_output = df[['APRESENTAÇÃO', 'Quantidade', 'Tipo de Produto', 'Detalhes da Embalagem']]
    df_output.to_csv("output2.csv", index=False)

if __name__ == "__main__":
    main()


In [7]:
import re
import pandas as pd

def process_text(text):
    # Converte a entrada para uma string
    text = str(text)
    
    # Retira os espaços extras
    text = text.strip()

    # Dicionário de formas farmacêuticas
    pharmaceutical_form = {
        "COM": "Comprimido",
        "COM REV": "Comprimido Revestido",
        "CAP DURA": "Cápsula Dura",
        "CAP MOLE": "Cápsula Mole",
        "POM DERM": "Pomada Dermatológica",
        "EMU INJ IV": "Emulsão Injetável Intravenosa",
        "SOL INFUS IV": "Solução para Infusão Intravenosa",
        "SOL INJ IV": "Solução Injetável Intravenosa",
        "SOL INJ": "Solução Injetável",
        "EMU INJ": "Emulsão Injetável",
        'CAP': 'Cápsula',
        'COMP': 'Comprimido',
        'SOL': 'Solução',
        'CREM': 'Creme',
        'GEL': 'Gel',
        'SUS': 'Suspensão',
        'XPE': 'Xarope',
        'IMPLANTE': 'Implante',
        'ANEL': 'Anel',
        'XAMP': 'Xampu',
        'CM': 'Comprimido Mastigável',
        'EMU': 'Emulsão',
        'LIÓF': 'Liofilizado',
        'DP': 'Diluente para',
        'INFUS': 'Infusão',
        'NEB': 'Nebulização',
        'TALQUEIRA': 'Talqueira',
        'GRAN': 'Granulado',
        "POM": "Pomada",
        "AER TOP": "Aerossol Tópico",
        "FIL OR": "Filme Oral",
        "COLUT": "Colutório",  
        "PO LIOF INJ": "Pó Liofilizado para Injeção",
        "LOC": "Loção",
        "ELX": "Elixir",
        "DRG": "Drágea",
        "SUP RET": "Supositório Retal",
        "ADES TRANSD": "Adesivo Transdérmico",
        "PO EFEV": "Pó Efervescente",
        "LIQ": "Líquido",
        "INAL": "Inalação",
        "PAS DURA": "Pastilha Dura",
        "LOÇÃO": "Loção",
        "PAST": "Pastilha",
        "PÓ LIOF INJ": "Pó Liofilizado para Injeção",
        "SHAMP": "Shampoo",
        "PO OR": "Pó Oral",
        "CR DERM": "Creme Dermatológico",
        "PÓ EFERV": "Pó Efervescente",
        "PÓ": "Pó",
        "OVL VAG": "Óvulo Vaginal",
        "OVL": "Óvulo",
        "DIU": "Dispositivo Intrauterino",
        "IMPL": "Implante",
        "ADES": "Adesivo",
        "LOÇ": "Loção",
        "LIN": "Linimento",
        "EMPL": "Emplastro",
        "CP DURA": "Comprimido de Liberação Prolongada",
        "PO EFEV": "Pó Efervescente",
        "PAS": "Pastilha",
        "PAS GOM": "Pastilha Gamosa",
        "PO LIOF": "Pó Liofilizado"
    }
    
    form = None
    for key, value in pharmaceutical_form.items():
        if key in text:
            form = value
            text = text.replace(key, "").strip()

    embalagens = {
        "FR": "Frasco",
        "AMP": "Ampola",
        "CT": "Cartela",
        "TUBO": "Tubo",
        "SACHÊ": "Sachê",
        "BLÍSTER": "Blíster",
        "BISNAGA": "Bisnaga",
        "FAIXA": "Faixa",
        "SERINGA": "Seringa",
        "ESTOJO": "Estojo",
        "FLAC": "Flaconete",
        "POTE": "Pote"
    }

    emb_type = None
    for key, value in embalagens.items():
        if key in text:
            emb_type = value
            text = text.replace(key, "").strip()

    # Separa quantidade e unidade
    if text[0] == '(':
        # Se o texto começar com "(", obtemos todo o texto dentro dos parênteses
        quantity = re.search(r'\(.*?\)', text).group()
        remaining_text = text.replace(quantity, '').strip()
        # Consideramos a próxima palavra após os parênteses como 'unit'
        unit = remaining_text.split()[0] if len(remaining_text.split()) > 0 else None
    elif text[0].isdigit():
        quantity, unit = text.split()[0], text.split()[1]
    else:
        quantity, unit = None, None

    # A descrição do item é tudo o que resta depois de remover "quantidade + unidade"
    item_description = text.replace(str(quantity), '').replace(str(unit), '').strip() if quantity and unit else text

    # Extraímos a unidade de "unidades"
    units = re.search(r'X\s(\d+)', text)
    units = units.group(1) if units else None

    # Verificamos se "COM REV" está presente na descrição do item
    com_rev = "Não"
    if "REV" in item_description:
        com_rev = "Sim"
        # Removemos "REV" da descrição do item
        item_description = item_description.replace("REV", "").strip()

    # Removemos "X" e o valor inteiro que segue da descrição do item
    item_description = re.sub(r'X\s*\d+', '', item_description)

    return pd.Series([quantity, unit, item_description, units, com_rev, form, emb_type])



def main():
    df = pd.read_excel("new_gov.xlsx")
    df[['Quantidade de dosagem', 'Unidade de dosagem', 'Descrição do item', 'Unidades', 'Com Revestimento', 'Forma Farmacêutica', 'Tipo de Embalagem']] = df['APRESENTAÇÃO'].apply(process_text)
    df_output = df[['APRESENTAÇÃO', 'Quantidade de dosagem', 'Unidade de dosagem', 'Descrição do item', 'Unidades', 'Com Revestimento', 'Forma Farmacêutica', 'Tipo de Embalagem']]
    df_output.to_csv("output22.csv", index=False)

if __name__ == "__main__":
    main()


In [1]:
import re
import pandas as pd
import json

def process_text(text):
    # Converte a entrada para uma string
    text = str(text)
    
    # Retira os espaços extras
    text = text.strip()

    # Dicionário de formas farmacêuticas
    pharmaceutical_form = {
        "COM": "Comprimido",
        "COM REV": "Comprimido Revestido",
        "CAP DURA": "Cápsula Dura",
        "CAP MOLE": "Cápsula Mole",
        "POM DERM": "Pomada Dermatológica",
        "EMU INJ IV": "Emulsão Injetável Intravenosa",
        "SOL INFUS IV": "Solução para Infusão Intravenosa",
        "SOL INJ IV": "Solução Injetável Intravenosa",
        "SOL INJ": "Solução Injetável",
        "EMU INJ": "Emulsão Injetável",
    }
    
    form = None
    for key, value in pharmaceutical_form.items():
        if key in text:
            form = value
            text = text.replace(key, "").strip()

    # Separa quantidade e unidade
    if text[0] == '(':
        # Se o texto começar com "(", obtemos todo o texto dentro dos parênteses
        quantity = re.search(r'\(.*?\)', text).group()
        remaining_text = text.replace(quantity, '').strip()
        # Consideramos a próxima palavra após os parênteses como 'unit'
        unit = remaining_text.split()[0] if len(remaining_text.split()) > 0 else None
    elif text[0].isdigit():
        quantity, unit = text.split()[0], text.split()[1]
    else:
        quantity, unit = None, None

    # A descrição do item é tudo o que resta depois de remover "quantidade + unidade"
    item_description = text.replace(str(quantity), '').replace(str(unit), '').strip() if quantity and unit else text

    # Extraímos a unidade de "unidades"
    units = re.search(r'X\s(\d+)', text)
    units = units.group(1) if units else None

    # Verificamos se "COM REV" está presente na descrição do item
    com_rev = "Não"
    if "REV" in item_description:
        com_rev = "Sim"
        # Removemos "REV" da descrição do item
        item_description = item_description.replace("REV", "").strip()

    # Removemos "X" e o valor inteiro que segue da descrição do item
    item_description = re.sub(r'X\s*\d+', '', item_description)

    # Cria o dicionário para JSON
    item_dict = {
        'peso': quantity,
        'tipo': form,
        'multiplo': units,
        'descricao': item_description,
        'revestimento': com_rev
    }
    
    # Converte para JSON e retorna
    return json.dumps(item_dict)

def main():
    df = pd.read_excel("new_gov.xlsx")
    df['JSON'] = df['APRESENTAÇÃO'].apply(process_text)
    df_output = df[['APRESENTAÇÃO', 'JSON']]
    df_output.to_csv("output.json", index=False)

if __name__ == "__main__":
    main()


In [1]:
def main():
    df = pd.read_excel("new_gov.xlsx")
    df['JSON'] = df['APRESENTAÇÃO'].apply(process_text)
    df_output = df[['APRESENTAÇÃO', 'JSON']]
    df_output.to_json("output2.json", orient="records", lines=True)

if __name__ == "__main__":
    main()


NameError: name 'pd' is not defined

In [3]:
import pandas as pd

# Carregar o arquivo CSV em um DataFrame
df = pd.read_csv('output16.csv')

# Escrever o DataFrame como um arquivo JSON
df.to_json('output2.json', orient='records')
